# visualize

In [1]:
#from IPython.display import display
import ipywidgets as widgets
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
#from itables import init_notebook_mode
#init_notebook_mode(all_interactive=True)

## Load data

In [1]:
!python create.py --distribution_type gaussian

(60000, 28)
(60000, 28, 28)


2022-07-08 15:39:17.714566: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-07-08 15:39:17.714864: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-07-08 15:39:21.453055: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'nvcuda.dll'; dlerror: nvcuda.dll not found
2022-07-08 15:39:21.453252: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-07-08 15:39:21.459001: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: 20-0011
2022-07-08 15:39:21.459268: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: 20-0011
2022-07-08 15:39:21.459699: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with 

In [2]:
train_ts_ds = np.load('data/train/time_series.npy')
val_ts_ds = np.load('data/valid/time_series.npy')
test_ts_ds = np.load('data/test/time_series.npy')

train_gaf_ds = np.load('data/train/gafs.npy')
val_gaf_ds = np.load('data/valid/gafs.npy')
test_gaf_ds = np.load('data/test/gafs.npy')

train_label = np.load('data/train/labels.npy')
val_label = np.load('data/valid/labels.npy')
test_label = np.load('data/test/labels.npy')

## Aim for the next implementation

In [3]:
#TODO: Als einfaches widget umsetzen
#TODO: Berechnung des GAF on the run
class DataVizualisation(widgets.HBox):

    def __init__(self,arr):
        """
        arr: np.array (Samples,time,signals)
        """
        if not len(arr.shape)==3:
            raise ValueError("Numpy array has to have exactly 3 dimensions")

        # 1) Init class + instances
        super().__init__()
        self.arr = arr

        # 2) Define components

        self.fig_ts = go.FigureWidget(layout=go.Layout(margin=dict(l=5,r=5,t=5,b=5),width=800))
        self.fig_gaf = go.FigureWidget(layout=go.Layout(margin=dict(l=5,r=5,t=5,b=5),width=800))

        self.children =[self.fig_ts,self.fig_gaf]


In [4]:
#DataVizualisation(np.expand_dims(train_ts_ds,axis=2))

## Implementation

In [5]:
# init figure widgets for ts and gaf
ts_len = test_ts_ds.shape[1]
ts_widget = go.FigureWidget(px.line(y=[None] * ts_len, range_y = [0,1]))
gaf_widget = go.FigureWidget(px.imshow(np.zeros((ts_len,ts_len)), zmin = 0, zmax= 1, color_continuous_scale = 'gray'))

In [6]:
# init feature label dataframe
label_df = pd.DataFrame(columns = ['step_start', 'step_end', 'step_width', 'step_amp'])

In [7]:
# dropdown menu for ds selection
ds_split_dd = widgets.Dropdown(
    options = [('Training', 0), ('Validation', 1), ('Test', 2)],
    value = 0,
    description='Split:'
)

In [8]:
# init df out for df visualization
df_out = widgets.Output()
with df_out:
    display(label_df)

In [9]:
# slider for ds indices
data_slider = widgets.IntSlider(
    value = 0,
    min = 0,
    max = train_label.shape[0] - 1,
    step = 1,
    description = 'Sample:',
    continous_update=False
)

In [10]:
# changes fig and df data with respect to slider val
def data_slider_change(change):
        with df_out:
            display(pd.DataFrame(label_df.loc[data_slider.value]), clear=True)
            
        ts_widget.data[0].y = ts_data[data_slider.value, :]
        gaf_widget.data[0].z = gaf_data[data_slider.value,:,:]

In [11]:
data_slider.observe(data_slider_change)

In [12]:
# drop down for data set selection
def ds_split_dd_change(change):
    global ts_data, gaf_data, label_data, label_df, df_out
    if ds_split_dd.value == 0:
        ts_data = train_ts_ds
        gaf_data = train_gaf_ds
        label_data = train_label
    if ds_split_dd.value == 1:
        ts_data = val_ts_ds
        gaf_data = val_gaf_ds
        label_data = val_label
    if ds_split_dd.value == 2:
        ts_data = test_ts_ds
        gaf_data = test_gaf_ds
        label_data = test_label
        
    label_df = label_df[0:0]
    label_df = label_df.append(pd.DataFrame(label_data, columns = label_df.columns))
    data_slider.value = 0
    data_slider.max = len(label_df) -1
    with df_out:
        display(label_df, clear=True)
        

In [13]:
ds_split_dd_change(None)

In [14]:
data_slider_change(None)

In [15]:
ds_split_dd.observe(ds_split_dd_change)

In [16]:
fig_box = widgets.HBox()

In [17]:
# radio buttons to choose between ts, gaf or ts and gaf figure presentation
fig_box_childs_rb = widgets.RadioButtons(
    options = [('TS', 0), ('GAF', 1), ('TS & GAF', 2)],
    value = 0,
    description='Figure:',
)

In [18]:
# set child of fig box with respect to chosen radiobutton
def fig_box_childs_rb_change(change):
    global fig_box
    if fig_box_childs_rb.value == 0:
        fig_box.children = [ts_widget]
    if fig_box_childs_rb.value == 1:
        fig_box.children = [gaf_widget]
    if fig_box_childs_rb.value == 2:
        fig_box.children = [ts_widget, gaf_widget]

In [19]:
fig_box_childs_rb_change(None)

In [20]:
fig_box_childs_rb.observe(fig_box_childs_rb_change)

In [21]:
a = [1,2,3,4,5,6]

In [22]:
val_gaf_ds.shape

(9600, 28, 28)

In [23]:
widgets.VBox([widgets.HBox([widgets.VBox([ds_split_dd, data_slider]), fig_box_childs_rb]), widgets.HBox([fig_box, df_out])])

In [24]:
# table fixen (und nur eine zeile bei ausgewähltem sample anzeigen)!!!!!!! sehr wichtig vor korrelation bestimmen
# in eigene notebooks:
# visualization of predicting time series  (test sample vis -> prediction decoder vis) (ipywidgets) + label daneben anzeigen (eine zeile aus der tabelle)
# visualisierung latent space (siehe colab) und andere vis-colab notebooks(ipywidgets)
# korrelation visualisieren
# r2 score auf zeitreihe (und bild)
# notebooks dokumentation (colab)
# -> alles bis montag pushen
# danach:
# cyclic beta -> statt callback einfach modell öfter trainieren mit unterschiedlichen beta
#  ggf reconstruction error erstmal vernachlässigen: hier die frage -> kann das modell gut clustern (gutes disentanglement)